### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
#Calculate totals
total_schools = school_data["school_name"].nunique()
total_students = student_data["Student ID"].nunique()
total_budget = school_data["budget"].sum()
avg_math_score = school_data_complete["math_score"].mean()
avg_reading_score = school_data_complete["reading_score"].mean()
overall_passing_rate = (avg_math_score + avg_reading_score)/2

#Calculate the percentage of students with a passing math score
math_passing = school_data_complete.apply(lambda x: True if x['math_score'] >= 70 else False , axis=1)
count = len(math_passing[math_passing == True].index)
passed_math = (count / total_students) * 100
#passed_math

#Calculate the percentage of students with a passing reading score
reading_passing = school_data_complete.apply(lambda x: True if x['reading_score'] >= 70 else False , axis=1)
count = len(reading_passing[reading_passing == True].index)
passed_reading = (count / total_students) * 100
#passed_reading

#Creating dataframe
district_summary = [{'Total Schools': total_schools, 'Total Students': total_students, 'Total Budget':total_budget,
                    'Average Math Score': avg_math_score, 'Average Reading Score':avg_reading_score, '% Passing Math':passed_math,
                    '% Passing Reading': passed_reading, '% Overall Passing Rate':overall_passing_rate}]
district_summary = pd.DataFrame(district_summary) 
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,80.431606


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [3]:
school_summary = pd.DataFrame()

school_summary['School Type'] = school_data_complete.groupby('school_name')['type'].first()
school_summary['Total Students'] = school_data_complete.groupby('school_name')['student_name'].count()
school_summary['Total School Budget'] = school_data_complete.groupby('school_name')['budget'].mean()
school_summary['Per Student Budget'] = school_summary['Total School Budget']/school_summary['Total Students']

school_summary['Average Math Score'] = school_data_complete.groupby('school_name')['math_score'].mean()
school_summary['Average Reading Score'] = school_data_complete.groupby('school_name')['reading_score'].mean()


school_students_passedmath = school_data_complete.groupby('school_name')['math_score'].apply(lambda x: (x >= 70).sum()).reset_index(name='count')
school_students_passedreading = school_data_complete.groupby('school_name')['reading_score'].apply(lambda x: (x >= 70).sum()).reset_index(name='count')

school_summary = pd.merge(school_summary, school_students_passedmath, on="school_name")
school_summary = pd.merge(school_summary, school_students_passedreading, on="school_name")

school_summary['% Passing Math'] = (school_summary['count_x']/school_summary['Total Students']) * 100
school_summary['% Passing Reading'] = (school_summary['count_y']/school_summary['Total Students']) * 100
school_summary['% Overall Passing Rate'] = (school_summary['% Passing Math']+school_summary['% Passing Reading'])/2

del school_summary['count_x']
del school_summary['count_y']

#school_students_passedreading
#school_summary.head()

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [4]:
top_performing_schools = school_summary.sort_values('% Overall Passing Rate',ascending = False).head(5)
top_performing_schools.set_index('school_name')

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [5]:
bottom_performing_schools = school_summary.sort_values('% Overall Passing Rate',ascending = True).head(5)
bottom_performing_schools.set_index('school_name')

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [6]:
math_scores_bygrade = pd.DataFrame()
math_scores_bygrade['school_name'] = school_data_complete.groupby(['school_name','grade'])['school_name'].first()
math_scores_bygrade['grade'] = school_data_complete.groupby(['school_name','grade'])['grade'].first()
math_scores_bygrade['score'] = school_data_complete.groupby(['school_name','grade'])['math_score'].mean()

math_scores_bygrade.set_index('school_name')
summary_math = math_scores_bygrade.pivot(index='school_name', columns='grade')
summary_math

score                                 
grade                       10th       11th       12th        9th
school_name                                                      
Bailey High School     76.996772  77.515588  76.492218  77.083676
Cabrera High School    83.154506  82.765560  83.277487  83.094697
Figueroa High School   76.539974  76.884344  77.151369  76.403037
Ford High School       77.672316  76.918058  76.179963  77.361345
Griffin High School    84.229064  83.842105  83.356164  82.044010
Hernandez High School  77.337408  77.136029  77.186567  77.438495
Holden High School     83.429825  85.000000  82.855422  83.787402
Huang High School      75.908735  76.446602  77.225641  77.027251
Johnson High School    76.691117  77.491653  76.863248  77.187857
Pena High School       83.372000  84.328125  84.121547  83.625455
Rodriguez High School  76.612500  76.395626  77.690748  76.859966
Shelton High School    82.917411  83.383495  83.778976  83.420755
Thomas High School     83.087886  83.498795  83.497041  83.590022
Wilson High School     83.724422  83.195326  83.035794  83.085578
Wright High School     84.010288  83.836782  83.644986  83.264706

## Reading Scores by Grade

* Perform the same operations as above for reading scores

In [7]:
reading_scores_bygrade = pd.DataFrame()
reading_scores_bygrade['school_name'] = school_data_complete.groupby(['school_name','grade'])['school_name'].first()
reading_scores_bygrade['grade'] = school_data_complete.groupby(['school_name','grade'])['grade'].first()
reading_scores_bygrade['score'] = school_data_complete.groupby(['school_name','grade'])['reading_score'].mean()

reading_scores_bygrade.set_index('school_name')
summary_reading = reading_scores_bygrade.pivot(index='school_name', columns='grade')
summary_reading



score                                 
grade                       10th       11th       12th        9th
school_name                                                      
Bailey High School     80.907183  80.945643  80.912451  81.303155
Cabrera High School    84.253219  83.788382  84.287958  83.676136
Figueroa High School   81.408912  80.640339  81.384863  81.198598
Ford High School       81.262712  80.403642  80.662338  80.632653
Griffin High School    83.706897  84.288089  84.013699  83.369193
Hernandez High School  80.660147  81.396140  80.857143  80.866860
Holden High School     83.324561  83.815534  84.698795  83.677165
Huang High School      81.512386  81.417476  80.305983  81.290284
Johnson High School    80.773431  80.616027  81.227564  81.260714
Pena High School       83.612000  84.335938  84.591160  83.807273
Rodriguez High School  80.629808  80.864811  80.376426  80.993127
Shelton High School    83.441964  84.373786  82.781671  84.122642
Thomas High School     84.254157  83.585542  83.831361  83.728850
Wilson High School     84.021452  83.764608  84.317673  83.939778
Wright High School     83.812757  84.156322  84.073171  83.833333

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [8]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [9]:
school_summary["Spending Ranges (Per Student)"] = pd.cut(school_summary["Per Student Budget"], spending_bins, labels=group_names)

In [10]:
school_spending = pd.DataFrame()
school_spending["Average Math Score"] = school_summary.groupby('Spending Ranges (Per Student)')['Average Math Score'].mean()
school_spending["Average Reading Score"] = school_summary.groupby('Spending Ranges (Per Student)')['Average Reading Score'].mean()
school_spending["% Passing Math"] = school_summary.groupby('Spending Ranges (Per Student)')['% Passing Math'].mean()
school_spending["% Passing Reading"] = school_summary.groupby('Spending Ranges (Per Student)')['% Passing Reading'].mean()
school_spending['% Overall Passing Rate'] = (school_spending['% Passing Math']+school_spending['% Passing Reading'])/2

school_spending

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [11]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names2 = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [12]:
school_summary['size'] = school_data_complete.groupby('School ID')['size'].first()
school_summary['School Size'] = pd.cut(school_summary['size'], size_bins, labels=group_names2)

In [13]:
school_size_summary = pd.DataFrame()
school_size_summary["Average Math Score"] = school_summary.groupby('School Size')['Average Math Score'].mean()
school_size_summary["Average Reading Score"] = school_summary.groupby('School Size')['Average Reading Score'].mean()
school_size_summary["% Passing Math"] = school_summary.groupby('School Size')['% Passing Math'].mean()
school_size_summary["% Passing Reading"] = school_summary.groupby('School Size')['% Passing Reading'].mean()
school_size_summary['% Overall Passing Rate'] = ( school_size_summary['% Passing Math'] + school_size_summary['% Passing Reading'] )/2

school_size_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),80.456190,82.505546,80.326073,88.584189,84.455131
Medium (1000-2000),80.878295,82.697890,82.317324,90.192458,86.254891
Large (2000-5000),80.148011,82.429660,80.320880,88.769335,84.545108


## Scores by School Type

* Perform the same operations as above, based on school type.

In [14]:
school_summary['School Type'] = school_data_complete.groupby('School ID')['type'].first()

In [15]:
school_type_summary = pd.DataFrame()
school_type_summary["Average Math Score"] = school_summary.groupby('School Type')['Average Math Score'].mean()
school_type_summary["Average Reading Score"] = school_summary.groupby('School Type')['Average Reading Score'].mean()
school_type_summary["% Passing Math"] = school_summary.groupby('School Type')['% Passing Math'].mean()
school_type_summary["% Passing Reading"] = school_summary.groupby('School Type')['% Passing Reading'].mean()
school_type_summary['% Overall Passing Rate'] = ( school_type_summary['% Passing Math'] + school_type_summary['% Passing Reading'] )/2

school_type_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,80.324201,82.429369,79.873967,88.624209,84.249088
District,80.556334,82.643266,82.259154,89.898811,86.078983
